In [1]:
import pandas as pd
from sklearn.linear_model import LinearRegression, ElasticNet
import numpy as np
from sklearn.metrics import r2_score 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import make_column_transformer 
from sklearn.compose import make_column_selector
import os
os.chdir(r"C:\Training\Kaggle\Competitions\Playground Competitions\UsedCarsPrices")

In [2]:
train = pd.read_csv("train.csv", index_col=0)
train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 188533 entries, 0 to 188532
Data columns (total 12 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         188533 non-null  object
 1   model         188533 non-null  object
 2   model_year    188533 non-null  int64 
 3   milage        188533 non-null  int64 
 4   fuel_type     183450 non-null  object
 5   engine        188533 non-null  object
 6   transmission  188533 non-null  object
 7   ext_col       188533 non-null  object
 8   int_col       188533 non-null  object
 9   accident      186081 non-null  object
 10  clean_title   167114 non-null  object
 11  price         188533 non-null  int64 
dtypes: int64(3), object(9)
memory usage: 18.7+ MB


In [3]:
X_train = train.drop('price', axis=1)
y_train = train['price']
ohe = OneHotEncoder(handle_unknown='infrequent_if_exist',
      sparse_output=False,min_frequency=3000).set_output(transform='pandas')
ct_ohe = make_column_transformer(('passthrough', make_column_selector(dtype_exclude=object)  ),
                             (ohe, make_column_selector(dtype_include=object) ),
                            verbose_feature_names_out=False).set_output(transform='pandas')
X_train_ohe = ct_ohe.fit_transform(X_train)

In [4]:
X_train_ohe.isnull().sum().sum()

0

In [5]:
X_test = pd.read_csv("test.csv", index_col=0)
X_test.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125690 entries, 188533 to 314222
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   brand         125690 non-null  object
 1   model         125690 non-null  object
 2   model_year    125690 non-null  int64 
 3   milage        125690 non-null  int64 
 4   fuel_type     122307 non-null  object
 5   engine        125690 non-null  object
 6   transmission  125690 non-null  object
 7   ext_col       125690 non-null  object
 8   int_col       125690 non-null  object
 9   accident      124058 non-null  object
 10  clean_title   111451 non-null  object
dtypes: int64(2), object(9)
memory usage: 11.5+ MB


In [6]:
X_test_ohe = ct_ohe.transform(X_test)
len(X_test_ohe.columns)

62

In [7]:
X_test_ohe.info()

<class 'pandas.core.frame.DataFrame'>
Index: 125690 entries, 188533 to 314222
Data columns (total 62 columns):
 #   Column                                               Non-Null Count   Dtype  
---  ------                                               --------------   -----  
 0   model_year                                           125690 non-null  int64  
 1   milage                                               125690 non-null  int64  
 2   brand_Audi                                           125690 non-null  float64
 3   brand_BMW                                            125690 non-null  float64
 4   brand_Cadillac                                       125690 non-null  float64
 5   brand_Chevrolet                                      125690 non-null  float64
 6   brand_Dodge                                          125690 non-null  float64
 7   brand_Ford                                           125690 non-null  float64
 8   brand_GMC                                            1

In [8]:
len(X_train_ohe.columns)

62

In [9]:
lr = LinearRegression()
lr.fit(X_train_ohe, y_train)
y_pred = lr.predict(X_test_ohe)

In [10]:
y_pred[y_pred<150] = 150

In [11]:
np.quantile(y_pred,[0,0.25,0.5,0.75,1])

array([   150.        ,  27161.03702497,  46878.36539031,  62641.23550645,
       109878.76745113])

In [12]:
submit = pd.read_csv("sample_submission.csv")
submit['price'] = y_pred
submit.to_csv("ss_5_nov_1_lr.csv", index=False)